## Thermodynamic Property Prediction for H, S, Tc
1. Generates models/predictions for H, S, Tc
2. Uses transfer learning to predict Tc
3. Calculates a prediction for Tc using H and S predictions and the Tc formula

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
import joblib
import os


from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.kernel_ridge import KernelRidge

from xgboost import XGBRegressor

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from great_tables import GT

In [5]:
plt.style.use("presentation.mplstyle")
mpl.rcParams['axes.prop_cycle'] = cycler(color=['#5c3c8b', '#92c36d', '#ee9432', '#496391', '#85a5cd', '#FDF3CC'])
plotsize = 5

### Define Parameters

In [6]:
file_info = {
    "enthalpy": "2_split_datasets/enthalpy_80_MI_reduced.csv",
    "entropy": "2_split_datasets/entropy_80_MI_reduced.csv",
    "tc": "2_split_datasets/has_both_101_MI_reduced.csv",
    "log_tc": "2_split_datasets/has_both_101_MI_reduced.csv", 
}

feature_sets = [
    (name, pd.read_csv(path, index_col=0))
    for name, path in file_info.items()
]

In [7]:
# store column indices for use in the loop
target_idx = 0
phase_split_idx = 1
category_idx = 2
smiles_idx = 3
global_test_idx = 4
first_feature_idx = 5

In [8]:
models = [
    ('gpr', GaussianProcessRegressor(), {
        "model__kernel": [], 
    }),
    ('rf', RandomForestRegressor(), {
        'model__n_estimators': [25, 50, 100],
        'model__max_depth': [None, 10],
        'model__min_samples_split': [2, 5]
    }),
    ('xgb', XGBRegressor(), {
        'model__n_estimators': [50, 100],
        'model__max_depth': [3, 6],
        'model__learning_rate': [0.01, 0.1],
        'model__subsample': [0.8, 1.0]
    }),
    ('svr', SVR(), {
        'model__C': [0.1, 1, 10],
        'model__gamma': ['scale', 'auto'],
        'model__kernel': ['rbf']
    }),
    ('krr', KernelRidge(), {
        'model__alpha': [1e-3, 1e-2, 1e-1, 1.0],
        'model__kernel': ['rbf', 'linear', 'poly'],
        'model__gamma': [0.01, 0.1, 1],
    }),
]

In [9]:
outer_splits = 10
inner_splits = 5
results = []

model_dir = "3_saved_models"
output_dir = "3_images_and_csvs"

os.makedirs(model_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

### Generate First Model Table

In [10]:
df = pd.read_csv(f"{output_dir}/model_results_summary.csv")

In [11]:
data = {
    "Model": [item.upper() for item in df['model'][:len(models)].tolist()],
    
    "Split by Chemistry.ΔHₚ.MAE": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.ΔHₚ.R²": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    "Split by Chemistry.ΔSₚ.MAE": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.ΔSₚ.R²": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    "Split by Chemistry.Tc.MAE": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.Tc.R²": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    "Split by Chemistry.log Tc.MAE": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.log Tc.R²": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    

    "Split by Phase.ΔHₚ.MAE": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.ΔHₚ.R²": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
    "Split by Phase.ΔSₚ.MAE": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.ΔSₚ.R²": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
    "Split by Phase.Tc.MAE": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.Tc.R²": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
    "Split by Phase.log Tc.MAE": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.log Tc.R²": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
}

gt = GT(pd.DataFrame(data)).cols_align(align="center")

for split in ['Chemistry', 'Phase']:
    for prop in ['ΔHₚ', 'ΔSₚ', 'Tc', 'log Tc']:
            for metric in ['MAE', 'R²']:
                gt = gt.fmt_number(columns=f"Split by {split}.{prop}.{metric}", decimals=2, drop_trailing_zeros=False)

gt = gt.fmt_integer(columns="Split by Chemistry.Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Phase.Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Chemistry.log Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Phase.log Tc.MAE", use_seps=False)

gt = gt.opt_horizontal_padding(scale=2.0)
gt = gt.opt_vertical_padding(scale=1.5)

gt = (gt.tab_options(container_width = "100%",column_labels_background_color="#C0ABDC",))

gt = gt.tab_options(
    column_labels_border_top_color="black",
    column_labels_border_top_style="solid",
    column_labels_border_top_width="2px",

    column_labels_border_bottom_color="black",
    column_labels_border_bottom_style="solid",
    column_labels_border_bottom_width="2px",

    table_body_border_bottom_color="black",
    table_body_border_bottom_style="solid",
    table_body_border_bottom_width="2px",
)

gt = gt.tab_spanner_delim()
gt.save(f"{output_dir}/model_result_table.png")

GT(_tbl_data=  Model  Split by Chemistry.ΔHₚ.MAE  Split by Chemistry.ΔHₚ.R²  \
0   GPR                        1.99                       0.85   
1    RF                        1.27                       0.91   
2   XGB                        1.48                       0.90   
3   SVR                        1.70                       0.87   
4   KRR                        1.78                       0.89   

   Split by Chemistry.ΔSₚ.MAE  Split by Chemistry.ΔSₚ.R²  \
0                        3.40                       0.76   
1                        3.51                       0.76   
2                        4.05                       0.69   
3                        3.24                       0.80   
4                        3.91                       0.66   

   Split by Chemistry.Tc.MAE  Split by Chemistry.Tc.R²  \
0                      239.0                     -0.72   
1                      288.0                     -3.09   
2                      194.0                      0.12   
3                      196.0                     -0.06   
4                      197.0                     -0.22   

   Split by Chemistry.log Tc.MAE  Split by Chemistry.log Tc.R²  \
0                         173.87                         -0.08   
1                         142.13                          0.23   
2                         203.79                         -0.72   
3                         143.17                          0.31   
4                         156.95                          0.09   

   Split by Phase.ΔHₚ.MAE  Split by Phase.ΔHₚ.R²  Split by Phase.ΔSₚ.MAE  \
0                    1.99                   0.85                    3.40   
1                    1.26                   0.90                    3.39   
2                    1.48                   0.90                    3.56   
3                    1.70                   0.87                    3.24   
4                    1.99                   0.82                    3.91   

   Split by Phase.ΔSₚ.R²  Split by Phase.Tc.MAE  Split by Phase.Tc.R²  \
0                   0.76                  239.0                 -0.72   
1                   0.77                  226.0                 -0.61   
2                   0.75                  194.0                  0.12   
3                   0.80                  196.0                 -0.06   
4                   0.66                  197.0                 -0.22   

   Split by Phase.log Tc.MAE  Split by Phase.log Tc.R²  
0                     173.87                     -0.08  
1                     160.17                      0.07  
2                     140.63                      0.20  
3                     143.17                      0.31  
4                     156.95                      0.09  , _body=<great_tables._gt_data.Body object at 0x302b80a10>, _boxhead=Boxhead([ColInfo(var='Model', type=<ColInfoTypeEnum.default: 1>, column_label='Model', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔHₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔHₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔSₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔSₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.Tc.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.Tc.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.log Tc.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.log Tc.R²', type=<ColInfoTypeEnum.defaul

### Add Transfer Learning to Table

In [12]:
best_H_model = None
best_S_model = None

for file in os.listdir("3_saved_models"):
    if "enthalpy" in file and "BEST" in file and "chemistry" in file:
        best_H_model = joblib.load(os.path.join("3_saved_models", file))
        print(file)
    elif "entropy" in file and "BEST" in file and "chemistry" in file:
        best_S_model = joblib.load(os.path.join("3_saved_models", file))
        print(file)

entropy_svr_chemistry_cluster_BEST.joblib
enthalpy_rf_chemistry_cluster_BEST.joblib


In [13]:
# enthalpy/entropy pred columns
entc = feature_sets[0][1].iloc[:, first_feature_idx:].columns
entrc = feature_sets[1][1].iloc[:, first_feature_idx:].columns

# make predictions
h_preds = best_H_model.predict(feature_sets[2][1].iloc[:, first_feature_idx:][entc])
s_preds = best_S_model.predict(feature_sets[2][1].iloc[:, first_feature_idx:][entrc])

# add to tc data set
tc_df = feature_sets[2][1].copy()
tc_df['predicted_H_TL'] = h_preds
tc_df['predicted_S_TL'] = s_preds
tc_df['predicted_TC_TL'] = h_preds*1000/s_preds - 273.15

feature_sets.append((f"{feature_sets[2][0]}_transfer", tc_df))

In [14]:
df_tl = pd.read_csv(os.path.join(output_dir, "transfer_model_results_summary.csv"))

In [15]:
# flag best Tc transfer model
for split_type in df_tl['split_type'].unique():
    for target_name in ['tc_transfer']:
        # find the best model with the lowest MAE
        subset = df_tl[(df_tl['split_type'] == split_type) & (df_tl['target_Name'] == target_name)]
        best_model_row = subset.sort_values(by='test_mae').head(1)
        
        if not best_model_row.empty:
            best_model_name = best_model_row['model'].values[0]
            model_path = f"{model_dir}/{target_name}_{best_model_name}_{split_type}_transfer_learning.joblib"
            if os.path.exists(model_path):
                new_model_path = f"{model_dir}/{target_name}_{best_model_name}_{split_type}_TL_BEST.joblib"
                os.rename(model_path, new_model_path)

In [16]:
data = {
    "Model": [item.upper() for item in df['model'][:len(models)].tolist()],
    
    "Split by Chemistry.ΔHₚ.MAE": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.ΔHₚ.R²": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    "Split by Chemistry.ΔSₚ.MAE": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.ΔSₚ.R²": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    "Split by Chemistry.Tc.MAE": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.Tc.R²": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    "Split by Chemistry.log Tc.MAE": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.log Tc.R²": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),

    "Split by Chemistry.Tc TL.MAE": df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Split by Chemistry.Tc TL.R²": df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),

    "Split by Phase.ΔHₚ.MAE": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.ΔHₚ.R²": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
    "Split by Phase.ΔSₚ.MAE": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.ΔSₚ.R²": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
    "Split by Phase.Tc.MAE": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.Tc.R²": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'phase')]['test_r2'].tolist(),
    "Split by Phase.log Tc.MAE": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.log Tc.R²": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'phase')]['test_r2'].tolist(),

    "Split by Phase.Tc TL.MAE": df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'phase')]['test_mae'].tolist(),
    "Split by Phase.Tc TL.R²": df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'phase')]['test_r2'].tolist(),
}

gt = GT(pd.DataFrame(data)).cols_align(align="center")

for split in ['Chemistry', 'Phase']:
    for prop in ['ΔHₚ', 'ΔSₚ', 'Tc', 'log Tc']:
            for metric in ['MAE', 'R²']:
                gt = gt.fmt_number(columns=f"Split by {split}.{prop}.{metric}", decimals=2, drop_trailing_zeros=False)

gt = gt.fmt_integer(columns="Split by Chemistry.Tc TL.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Phase.Tc TL.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Chemistry.Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Phase.Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Chemistry.Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Phase.Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Chemistry.log Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Split by Phase.log Tc.MAE", use_seps=False)

gt = gt.opt_horizontal_padding(scale=2.0)
gt = gt.opt_vertical_padding(scale=1.5)

gt = (gt.tab_options(container_width = "100%",column_labels_background_color="#C0ABDC",))

gt = gt.tab_options(
    column_labels_border_top_color="black",
    column_labels_border_top_style="solid",
    column_labels_border_top_width="2px",

    column_labels_border_bottom_color="black",
    column_labels_border_bottom_style="solid",
    column_labels_border_bottom_width="2px",

    table_body_border_bottom_color="black",
    table_body_border_bottom_style="solid",
    table_body_border_bottom_width="2px",
) 

gt = gt.tab_spanner_delim()
gt.save(f"{output_dir}/transfer_model_result_table.png")

GT(_tbl_data=  Model  Split by Chemistry.ΔHₚ.MAE  Split by Chemistry.ΔHₚ.R²  \
0   GPR                        1.99                       0.85   
1    RF                        1.27                       0.91   
2   XGB                        1.48                       0.90   
3   SVR                        1.70                       0.87   
4   KRR                        1.78                       0.89   

   Split by Chemistry.ΔSₚ.MAE  Split by Chemistry.ΔSₚ.R²  \
0                        3.40                       0.76   
1                        3.51                       0.76   
2                        4.05                       0.69   
3                        3.24                       0.80   
4                        3.91                       0.66   

   Split by Chemistry.Tc.MAE  Split by Chemistry.Tc.R²  \
0                      239.0                     -0.72   
1                      288.0                     -3.09   
2                      194.0                      0.12   
3                      196.0                     -0.06   
4                      197.0                     -0.22   

   Split by Chemistry.log Tc.MAE  Split by Chemistry.log Tc.R²  \
0                         173.87                         -0.08   
1                         142.13                          0.23   
2                         203.79                         -0.72   
3                         143.17                          0.31   
4                         156.95                          0.09   

   Split by Chemistry.Tc TL.MAE  ...  Split by Phase.ΔHₚ.MAE  \
0                        239.46  ...                    1.99   
1                        331.64  ...                    1.26   
2                        255.51  ...                    1.48   
3                        150.67  ...                    1.70   
4                        225.54  ...                    1.99   

   Split by Phase.ΔHₚ.R²  Split by Phase.ΔSₚ.MAE  Split by Phase.ΔSₚ.R²  \
0                   0.85                    3.40                   0.76   
1                   0.90                    3.39                   0.77   
2                   0.90                    3.56                   0.75   
3                   0.87                    3.24                   0.80   
4                   0.82                    3.91                   0.66   

   Split by Phase.Tc.MAE  Split by Phase.Tc.R²  Split by Phase.log Tc.MAE  \
0                  239.0                 -0.72                     173.87   
1                  226.0                 -0.61                     160.17   
2                  194.0                  0.12                     140.63   
3                  196.0                 -0.06                     143.17   
4                  197.0                 -0.22                     156.95   

   Split by Phase.log Tc.R²  Split by Phase.Tc TL.MAE  Split by Phase.Tc TL.R²  
0                     -0.08                    239.46                    -0.56  
1                      0.07                    335.78                   -12.15  
2                      0.20                    207.29                    -1.53  
3                      0.31                    150.67                     0.39  
4                      0.09                    225.54                    -3.99  

[5 rows x 21 columns], _body=<great_tables._gt_data.Body object at 0x3056b16d0>, _boxhead=Boxhead([ColInfo(var='Model', type=<ColInfoTypeEnum.default: 1>, column_label='Model', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔHₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔHₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔSₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry.ΔSₚ.R²', type=<ColInfoTypeE

### Chem-only table

In [17]:
data = {
    "Model": [item.upper() for item in df['model'][:len(models)].tolist()],
    
    "ΔHₚ.MAE": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "ΔHₚ.R²": df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    "ΔSₚ.MAE": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "ΔSₚ.R²": df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    "Tc.MAE": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Tc.R²": df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
    "log Tc.MAE": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "log Tc.R²": df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),

    "Tc TL.MAE": df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'chemistry_cluster')]['test_mae'].tolist(),
    "Tc TL.R²": df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'chemistry_cluster')]['test_r2'].tolist(),
}

gt = GT(pd.DataFrame(data)).cols_align(align="center")

for prop in ['ΔHₚ', 'ΔSₚ', 'Tc', 'log Tc', 'Tc TL']:
        for metric in ['MAE', 'R²']:
            gt = gt.fmt_number(columns=f"{prop}.{metric}", decimals=2, drop_trailing_zeros=False)

gt = gt.fmt_integer(columns="Tc TL.MAE", use_seps=False)
gt = gt.fmt_integer(columns="Tc.MAE", use_seps=False)
gt = gt.fmt_integer(columns="log Tc.MAE", use_seps=False)

gt = gt.opt_horizontal_padding(scale=2.0)
gt = gt.opt_vertical_padding(scale=1.5)

gt = (gt.tab_options(container_width = "100%",column_labels_background_color="#BDDAA7",))

gt = gt.tab_options(
    column_labels_border_top_color="black",
    column_labels_border_top_style="solid",
    column_labels_border_top_width="2px",

    column_labels_border_bottom_color="black",
    column_labels_border_bottom_style="solid",
    column_labels_border_bottom_width="2px",

    table_body_border_bottom_color="black",
    table_body_border_bottom_style="solid",
    table_body_border_bottom_width="2px",
)

gt = gt.tab_spanner_delim()
gt.save(f"{output_dir}/transfer_model_result_table_chem_only.png")

GT(_tbl_data=  Model  ΔHₚ.MAE  ΔHₚ.R²  ΔSₚ.MAE  ΔSₚ.R²  Tc.MAE  Tc.R²  log Tc.MAE  \
0   GPR     1.99    0.85     3.40    0.76   239.0  -0.72      173.87   
1    RF     1.27    0.91     3.51    0.76   288.0  -3.09      142.13   
2   XGB     1.48    0.90     4.05    0.69   194.0   0.12      203.79   
3   SVR     1.70    0.87     3.24    0.80   196.0  -0.06      143.17   
4   KRR     1.78    0.89     3.91    0.66   197.0  -0.22      156.95   

   log Tc.R²  Tc TL.MAE  Tc TL.R²  
0      -0.08     239.46     -0.56  
1       0.23     331.64    -12.55  
2      -0.72     255.51     -5.04  
3       0.31     150.67      0.39  
4       0.09     225.54     -3.99  , _body=<great_tables._gt_data.Body object at 0x302b63980>, _boxhead=Boxhead([ColInfo(var='Model', type=<ColInfoTypeEnum.default: 1>, column_label='Model', column_align='center', column_width=None), ColInfo(var='ΔHₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='ΔHₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='ΔSₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='ΔSₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Tc.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Tc.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='log Tc.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='log Tc.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Tc TL.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Tc TL.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x3056b38f0>, _spanners=Spanners([SpannerInfo(spanner_id='ΔHₚ', spanner_level=0, spanner_label='ΔHₚ', spanner_units=None, spanner_pattern=None, vars=['ΔHₚ.MAE', 'ΔHₚ.R²'], built=None), SpannerInfo(spanner_id='ΔSₚ', spanner_level=0, spanner_label='ΔSₚ', spanner_units=None, spanner_pattern=None, vars=['ΔSₚ.MAE', 'ΔSₚ.R²'], built=None), SpannerInfo(spanner_id='Tc', spanner_level=0, spanner_label='Tc', spanner_units=None, spanner_pattern=None, vars=['Tc.MAE', 'Tc.R²'], built=None), SpannerInfo(spanner_id='log Tc', spanner_level=0, spanner_label='log Tc', spanner_units=None, spanner_pattern=None, vars=['log Tc.MAE', 'log Tc.R²'], built=None), SpannerInfo(spanner_id='Tc TL', spanner_level=0, spanner_label='Tc TL', spanner_units=None, spanner_pattern=None, vars=['Tc TL.MAE', 'Tc TL.R²'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x305748920>, _formats=[<great_tables._gt_data.FormatInfo object at 0x3056f0b90>, <great_tables._gt_data.FormatInfo object at 0x3057480e0>, <great_tables._gt_data.FormatInfo object at 0x3057482f0>, <great_tables._gt_data.FormatInfo object at 0x305748c80>, <great_tables._gt_data.FormatInfo object at 0x30574b950>, <great_tables._gt_data.FormatInfo object at 0x3057484a0>, <great_tables._gt_data.FormatInfo object at 0x3057486e0>, <great_tables._gt_data.FormatInfo object at 0x305748590>, <great_tables._gt_data.FormatInfo object at 0x305748560>, <great_tables._gt_data.FormatInfo object at 0x305748530>, <great_tables._gt_data.FormatInfo object at 0x305748140>, <great_tables._gt_data.FormatInfo object at 0x3057487a0>, <great_tables._gt_data.FormatInfo object at 0x30574a930>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=Op

### Nested CV

In [18]:
data = {
    "Model": [item.upper() for item in df['model'][:len(models)].tolist()],
    
    "Split by Chemistry - Nested CV.ΔHₚ.MAE": [f"{m:0.2f} ± {s:0.2f}" for m, s in zip(df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_std'].tolist())],
    "Split by Chemistry - Nested CV.ΔHₚ.R²": [f"{m:0.2f} ± {s:0.2f}" for m, s in zip(df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_std'].tolist())],
    "Split by Chemistry - Nested CV.ΔSₚ.MAE": [f"{m:0.2f} ± {s:0.2f}" for m, s in zip(df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_std'].tolist())],
    "Split by Chemistry - Nested CV.ΔSₚ.R²": [f"{m:0.2f} ± {s:0.2f}" for m, s in zip(df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_std'].tolist())],
    "Split by Chemistry - Nested CV.Tc.MAE": [f"{m:.0f} ± {s:.0f}" for m, s in zip(df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_std'].tolist())],
    "Split by Chemistry - Nested CV.Tc.R²": [f"{m:.2f} ± {s:.2f}" for m, s in zip(df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_r2_mean'].tolist(), 
                                                                           df[(df['target_Name'] == 'tc') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_r2_std'].tolist())],
    "Split by Chemistry - Nested CV.log Tc.MAE": [f"{m:.0f} ± {s:.0f}" for m, s in zip(df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_std'].tolist())],
    "Split by Chemistry - Nested CV.log Tc.R²": [f"{m:.2f} ± {s:.2f}" for m, s in zip(df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_r2_mean'].tolist(), 
                                                                           df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'chemistry_cluster')]['nested_cv_r2_std'].tolist())],
    "Split by Chemistry - Nested CV.Tc TL.MAE": [f"{m:.0f} ± {s:.0f}" for m, s in zip(df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'chemistry_cluster')]['nested_cv_mae'].tolist(), 
                                                                           df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'chemistry_cluster')]['nested_cv_std'].tolist())],
    "Split by Chemistry - Nested CV.Tc TL.R²": [f"{m:.2f} ± {s:.2f}" for m, s in zip(df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'chemistry_cluster')]['nested_cv_r2_mean'].tolist(), 
                                                                           df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'chemistry_cluster')]['nested_cv_r2_std'].tolist())],
}

gt = GT(pd.DataFrame(data)).cols_align(align="center")

gt = gt.opt_horizontal_padding(scale=2.0)
gt = gt.opt_vertical_padding(scale=1.5)

gt = (gt.tab_options(container_width = "100%",column_labels_background_color="#C0ABDC",))

gt = gt.tab_options(
    column_labels_border_top_color="black",
    column_labels_border_top_style="solid",
    column_labels_border_top_width="2px",

    column_labels_border_bottom_color="black",
    column_labels_border_bottom_style="solid",
    column_labels_border_bottom_width="2px",

    table_body_border_bottom_color="black",
    table_body_border_bottom_style="solid",
    table_body_border_bottom_width="2px",
) 

gt = gt.tab_spanner_delim()
gt.save(f"{output_dir}/nested_CV_table_chem.png")

GT(_tbl_data=  Model Split by Chemistry - Nested CV.ΔHₚ.MAE  \
0   GPR                            2.88 ± 0.50   
1    RF                            2.83 ± 0.55   
2   XGB                            3.01 ± 0.69   
3   SVR                            3.06 ± 0.50   
4   KRR                            3.35 ± 1.42   

  Split by Chemistry - Nested CV.ΔHₚ.R²  \
0                           2.88 ± 0.50   
1                           2.83 ± 0.55   
2                           3.01 ± 0.69   
3                           3.06 ± 0.50   
4                           3.35 ± 1.42   

  Split by Chemistry - Nested CV.ΔSₚ.MAE  \
0                            4.15 ± 0.60   
1                            4.15 ± 0.67   
2                            4.25 ± 0.83   
3                            4.66 ± 0.89   
4                            7.60 ± 9.45   

  Split by Chemistry - Nested CV.ΔSₚ.R² Split by Chemistry - Nested CV.Tc.MAE  \
0                           4.15 ± 0.60                             390 ± 210   
1                           4.15 ± 0.67                             443 ± 255   
2                           4.25 ± 0.83                             438 ± 270   
3                           4.66 ± 0.89                             371 ± 199   
4                           7.60 ± 9.45                             400 ± 233   

  Split by Chemistry - Nested CV.Tc.R²  \
0                         -0.11 ± 0.12   
1                         -0.35 ± 0.58   
2                         -0.22 ± 0.27   
3                         -0.09 ± 0.10   
4                         -0.17 ± 0.18   

  Split by Chemistry - Nested CV.log Tc.MAE  \
0                                 365 ± 203   
1                                 362 ± 196   
2                                 352 ± 198   
3                                 387 ± 217   
4                                 445 ± 321   

  Split by Chemistry - Nested CV.log Tc.R²  \
0                             -0.08 ± 0.14   
1                             -0.09 ± 0.15   
2                             -0.01 ± 0.22   
3                             -0.50 ± 0.96   
4                             -1.34 ± 3.09   

  Split by Chemistry - Nested CV.Tc TL.MAE  \
0                                340 ± 216   
1                                325 ± 211   
2                                355 ± 200   
3                                326 ± 211   
4                                271 ± 232   

  Split by Chemistry - Nested CV.Tc TL.R²  
0                             0.03 ± 0.20  
1                            -0.08 ± 0.43  
2                            -0.64 ± 1.71  
3                             0.07 ± 0.16  
4                             0.40 ± 0.39  , _body=<great_tables._gt_data.Body object at 0x3056c7ec0>, _boxhead=Boxhead([ColInfo(var='Model', type=<ColInfoTypeEnum.default: 1>, column_label='Model', column_align='center', column_width=None), ColInfo(var='Split by Chemistry - Nested CV.ΔHₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry - Nested CV.ΔHₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Chemistry - Nested CV.ΔSₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry - Nested CV.ΔSₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Chemistry - Nested CV.Tc.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry - Nested CV.Tc.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Chemistry - Nested CV.log Tc.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Chemistry - Nested CV.log Tc.R²', type=<Co

In [19]:
data = {
    "Model": [item.upper() for item in df['model'][:len(models)].tolist()],
    
    "Split by Phase - Nested CV.ΔHₚ.MAE": [f"{m:0.2f} ± {s:0.2f}" for m, s in zip(df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'phase')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'phase')]['nested_cv_std'].tolist())],
    "Split by Phase - Nested CV.ΔHₚ.R²": [f"{m:0.2f} ± {s:0.2f}" for m, s in zip(df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'phase')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'enthalpy') & (df['split_type'] == 'phase')]['nested_cv_std'].tolist())],
    "Split by Phase - Nested CV.ΔSₚ.MAE": [f"{m:0.2f} ± {s:0.2f}" for m, s in zip(df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'phase')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'phase')]['nested_cv_std'].tolist())],
    "Split by Phase - Nested CV.ΔSₚ.R²": [f"{m:0.2f} ± {s:0.2f}" for m, s in zip(df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'phase')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'entropy') & (df['split_type'] == 'phase')]['nested_cv_std'].tolist())],
    "Split by Phase - Nested CV.Tc.MAE": [f"{m:.0f} ± {s:.0f}" for m, s in zip(df[(df['target_Name'] == 'tc') & (df['split_type'] == 'phase')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'tc') & (df['split_type'] == 'phase')]['nested_cv_std'].tolist())],
    "Split by Phase - Nested CV.Tc.R²": [f"{m:.2f} ± {s:.2f}" for m, s in zip(df[(df['target_Name'] == 'tc') & (df['split_type'] == 'phase')]['nested_cv_r2_mean'].tolist(), 
                                                                           df[(df['target_Name'] == 'tc') & (df['split_type'] == 'phase')]['nested_cv_r2_std'].tolist())],
    "Split by Phase - Nested CV.log Tc.MAE": [f"{m:.0f} ± {s:.0f}" for m, s in zip(df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'phase')]['nested_cv_mae'].tolist(), 
                                                                           df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'phase')]['nested_cv_std'].tolist())],
    "Split by Phase - Nested CV.log Tc.R²": [f"{m:.2f} ± {s:.2f}" for m, s in zip(df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'phase')]['nested_cv_r2_mean'].tolist(), 
                                                                           df[(df['target_Name'] == 'log_tc') & (df['split_type'] == 'phase')]['nested_cv_r2_std'].tolist())],
    "Split by Phase - Nested CV.Tc TL.MAE": [f"{m:.0f} ± {s:.0f}" for m, s in zip(df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'phase')]['nested_cv_mae'].tolist(), 
                                                                           df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'phase')]['nested_cv_std'].tolist())],
    "Split by Phase - Nested CV.Tc TL.R²": [f"{m:.2f} ± {s:.2f}" for m, s in zip(df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'phase')]['nested_cv_r2_mean'].tolist(), 
                                                                           df_tl[(df_tl['target_Name'] == 'tc_transfer') & (df_tl['split_type'] == 'phase')]['nested_cv_r2_std'].tolist())],
}

gt = GT(pd.DataFrame(data)).cols_align(align="center")

gt = gt.opt_horizontal_padding(scale=2)
gt = gt.opt_vertical_padding(scale=1.5)

gt = (gt.tab_options(container_width = "100%",column_labels_background_color="#C0ABDC",))

gt = gt.tab_options(
    column_labels_border_top_color="black",
    column_labels_border_top_style="solid",
    column_labels_border_top_width="2px",

    column_labels_border_bottom_color="black",
    column_labels_border_bottom_style="solid",
    column_labels_border_bottom_width="2px",

    table_body_border_bottom_color="black",
    table_body_border_bottom_style="solid",
    table_body_border_bottom_width="2px",
) 

gt = gt.tab_spanner_delim()
gt.save(f"{output_dir}/nested_CV_table_phase.png")

GT(_tbl_data=  Model Split by Phase - Nested CV.ΔHₚ.MAE Split by Phase - Nested CV.ΔHₚ.R²  \
0   GPR                        2.38 ± 0.40                       2.38 ± 0.40   
1    RF                        2.39 ± 0.47                       2.39 ± 0.47   
2   XGB                        2.44 ± 0.41                       2.44 ± 0.41   
3   SVR                        2.65 ± 0.42                       2.65 ± 0.42   
4   KRR                        2.48 ± 0.51                       2.48 ± 0.51   

  Split by Phase - Nested CV.ΔSₚ.MAE Split by Phase - Nested CV.ΔSₚ.R²  \
0                        4.05 ± 0.43                       4.05 ± 0.43   
1                        3.94 ± 0.33                       3.94 ± 0.33   
2                        4.00 ± 0.39                       4.00 ± 0.39   
3                        4.20 ± 0.42                       4.20 ± 0.42   
4                        7.02 ± 8.88                       7.02 ± 8.88   

  Split by Phase - Nested CV.Tc.MAE Split by Phase - Nested CV.Tc.R²  \
0                         363 ± 128                      0.07 ± 0.16   
1                         391 ± 142                     -0.29 ± 0.66   
2                         381 ± 136                      0.09 ± 0.18   
3                         369 ± 128                     -0.03 ± 0.05   
4                         375 ± 138                     -0.05 ± 0.29   

  Split by Phase - Nested CV.log Tc.MAE Split by Phase - Nested CV.log Tc.R²  \
0                             360 ± 131                          0.02 ± 0.12   
1                             352 ± 142                          0.05 ± 0.13   
2                             331 ± 131                          0.14 ± 0.19   
3                             354 ± 131                         -0.00 ± 0.26   
4                             367 ± 133                         -0.12 ± 0.55   

  Split by Phase - Nested CV.Tc TL.MAE Split by Phase - Nested CV.Tc TL.R²  
0                            334 ± 148                         0.07 ± 0.11  
1                            290 ± 177                         0.21 ± 0.43  
2                            303 ± 157                         0.26 ± 0.28  
3                            326 ± 148                         0.08 ± 0.12  
4                            259 ± 128                         0.14 ± 0.52  , _body=<great_tables._gt_data.Body object at 0x304099340>, _boxhead=Boxhead([ColInfo(var='Model', type=<ColInfoTypeEnum.default: 1>, column_label='Model', column_align='center', column_width=None), ColInfo(var='Split by Phase - Nested CV.ΔHₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Phase - Nested CV.ΔHₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Phase - Nested CV.ΔSₚ.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Phase - Nested CV.ΔSₚ.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Phase - Nested CV.Tc.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Phase - Nested CV.Tc.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Phase - Nested CV.log Tc.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Phase - Nested CV.log Tc.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None), ColInfo(var='Split by Phase - Nested CV.Tc TL.MAE', type=<ColInfoTypeEnum.default: 1>, column_label='MAE', column_align='center', column_width=None), ColInfo(var='Split by Phase - Nested CV.Tc TL.R²', type=<ColInfoTypeEnum.default: 1>, column_label='R²', column_align='center', column_width=None)]), _stub=<